<a href="https://colab.research.google.com/github/Mubarok982/DM_ML/blob/main/TUGAS_ML_Pertemuan6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Import Library

In [116]:
#Import Library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report

# 2. Load Dataset

In [117]:
#Load Dataset
data = pd.read_csv('Customer_support_data.csv')
data.head()

/usr/local/lib/python3.11/dist-packages/google/colab/_dataframe_summarizer.py:88: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  cast_date_col = pd.to_datetime(column, errors="coerce")


,Unique id,channel_name,category,Sub-category,Customer Remarks,Order_id,order_date_time,Issue_reported at,issue_responded,Survey_response_Date,Customer_City,Product_category,Item_price,connected_handling_time,Agent_name,Supervisor,Manager,Tenure Bucket,Agent Shift,CSAT Score
0,7e9ae164-6a8b-4521-a2d4-58f7c9fff13f,Outcall,Product Queries,Life Insurance,NaN,c27c9bb4-fa36-4140-9f1f-21009254ffdb,NaN,01/08/2023 11:13,01/08/2023 11:47,01-Aug-23,NaN,NaN,NaN,NaN,Richard Buchanan,Mason Gupta,Jennifer Nguyen,On Job Training,Morning,5
1,b07ec1b0-f376-43b6-86df-ec03da3b2e16,Outcall,Product Queries,Product Specific Information,NaN,d406b0c7-ce17-4654-b9de-f08d421254bd,NaN,01/08/2023 12:52,01/08/2023 12:54,01-Aug-23,NaN,NaN,NaN,NaN,Vicki Collins,Dylan Kim,Michael Lee,>90,Morning,5
2,200814dd-27c7-4149-ba2b-bd3af3092880,Inbound,Order Related,Installation/demo,NaN,c273368d-b961-44cb-beaf-62d6fd6c00d5,NaN,01/08/2023 20:16,01/08/2023 20:38,01-Aug-23,NaN,NaN,NaN,NaN,Duane Norman,Jackson Park,William Kim,On Job Training,Evening,5
3,eb0d3e53-c1ca-42d3-8486-e42c8d622135,Inbound,Returns,Reverse Pickup Enquiry,NaN,5aed0059-55a4-4ec6-bb54-97942092020a,NaN,01/08/2023 20:56,01/08/2023 21:16,01-Aug-23,NaN,NaN,NaN,NaN,Patrick Flores,Olivia Wang,John Smith,>90,Evening,5
4,ba903143-1e54-406c-b969-46c52f92e5df,Inbound,Cancellation,Not Needed,NaN,e8bed5a9-6933-4aff-9dc6-ccefd7dcde59,NaN,01/08/2023 10:30,01/08/2023 10:32,01-Aug-23,NaN,NaN,NaN,NaN,Christopher Sanchez,Austin Johnson,Michael Lee,0-30,Morning,5


# 3. Analisis Dataset

## 3.1 Memahami struktur data

In [118]:
# Memahami Struktur Data
print("\n--- Struktur Data ---")
print(f"Jumlah baris dan kolom: {data.shape}")
print(f"Tipe data setiap kolom:\n{data.dtypes}")
print("\n--- Lima Baris Pertama ---")
print(data.head())


--- Struktur Data ---
Jumlah baris dan kolom: (85907, 20)
Tipe data setiap kolom:
Unique id                   object
channel_name                object
category                    object
Sub-category                object
Customer Remarks            object
Order_id                    object
order_date_time             object
Issue_reported at           object
issue_responded             object
Survey_response_Date        object
Customer_City               object
Product_category            object
Item_price                 float64
connected_handling_time    float64
Agent_name                  object
Supervisor                  object
Manager                     object
Tenure Bucket               object
Agent Shift                 object
CSAT Score                   int64
dtype: object

--- Lima Baris Pertama ---
                              Unique id channel_name         category  \
0  7e9ae164-6a8b-4521-a2d4-58f7c9fff13f      Outcall  Product Queries   
1  b07ec1b0-f376-43b6-86df-ec

## 3.2 Menganalisis Data yang Hilang

In [119]:
# Menganalisis Data yang Hilang
print("\n--- Data yang Hilang ---")
missing_values = data.isnull().sum()
missing_percentage = (missing_values / len(data)) * 100
print(pd.DataFrame({'Jumlah Data Hilang': missing_values, 'Persentase': missing_percentage}))



--- Data yang Hilang ---
                         Jumlah Data Hilang  Persentase
Unique id                                 0    0.000000
channel_name                              0    0.000000
category                                  0    0.000000
Sub-category                              0    0.000000
Customer Remarks                      57165   66.542889
Order_id                              18232   21.222950
order_date_time                       68693   79.962052
Issue_reported at                         0    0.000000
issue_responded                           0    0.000000
Survey_response_Date                      0    0.000000
Customer_City                         68828   80.119199
Product_category                      68711   79.983005
Item_price                            68701   79.971364
connected_handling_time               85665   99.718300
Agent_name                                0    0.000000
Supervisor                                0    0.000000
Manager               

## 3.3 Menangani data yang hilang

In [120]:
# 1. Menghapus kolom dengan data hilang sangat besar (>70%)
kolom_dihapus = ['connected_handling_time', 'order_date_time', 'Product_category', 'Item_price']
kolom_dihapus = [kolom for kolom in kolom_dihapus if kolom in data.columns]
data = data.drop(columns=kolom_dihapus)

# 2. Mengisi nilai hilangCustomer Remarks (kategorikal): ganti dengan 'Tidak Diketahui'
data['Customer Remarks'] = data['Customer Remarks'].fillna('Tidak Diketahui')

# 3.CSAT Score (numerik): ganti dengan median
data['CSAT Score'] = data['CSAT Score'].fillna(data['CSAT Score'].median())

# 4. Mengisi dengan forward fill untuk sisa kolom
data = data.ffill()

# 4. Menghapus baris yang memiliki lebih dari satu nilai yang hilang (jika masih ada)
data = data.dropna(thresh=len(data.columns)-1)

# 5. Menggunakan imputasi
imputer = SimpleImputer(strategy='most_frequent')
for col in data.columns:
    if data[col].isnull().any():
        data[[col]] = imputer.fit_transform(data[[col]])

# 6. Menangani kolom waktu
tanggal_kolom = ['Survey_response_Date', 'Issue_reported at', 'issue_responded']
for kolom in tanggal_kolom:
    data[kolom] = pd.to_datetime(data[kolom], errors='coerce')

# 7. Verifikasi hasil
print("--- Jumlah Nilai Hilang Setelah Penanganan ---")
print(data.isnull().sum())

<ipython-input-120-88d34388a874>:27: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data[kolom] = pd.to_datetime(data[kolom], errors='coerce')


--- Jumlah Nilai Hilang Setelah Penanganan ---
Unique id                   0
channel_name                0
category                    0
Sub-category                0
Customer Remarks            0
Order_id                    0
Issue_reported at       53933
issue_responded         54022
Survey_response_Date        0
Customer_City               0
Agent_name                  0
Supervisor                  0
Manager                     0
Tenure Bucket               0
Agent Shift                 0
CSAT Score                  0
dtype: int64


# 3.4 Menghapus kolom numerik dan mengisi nilai kosong

In [121]:
data = data.drop(columns=['Issue_reported at', 'issue_responded'])
most_common_city = data['Customer_City'].mode()[0]
data['Customer_City'] = data['Customer_City'].fillna(most_common_city)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85907 entries, 0 to 85906
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Unique id             85907 non-null  object        
 1   channel_name          85907 non-null  object        
 2   category              85907 non-null  object        
 3   Sub-category          85907 non-null  object        
 4   Customer Remarks      85907 non-null  object        
 5   Order_id              85907 non-null  object        
 6   Survey_response_Date  85907 non-null  datetime64[ns]
 7   Customer_City         85907 non-null  object        
 8   Agent_name            85907 non-null  object        
 9   Supervisor            85907 non-null  object        
 10  Manager               85907 non-null  object        
 11  Tenure Bucket         85907 non-null  object        
 12  Agent Shift           85907 non-null  object        
 13  CSAT Score      

# 4. Encoding dan Modeling data Untuk naive bayes

## 4.1 a. Buat Kolom Label: CSAT_Label

In [122]:
data['CSAT_Label'] = data['CSAT Score'].apply(lambda x: 'Low' if x <= 3 else 'High')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85907 entries, 0 to 85906
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Unique id             85907 non-null  object        
 1   channel_name          85907 non-null  object        
 2   category              85907 non-null  object        
 3   Sub-category          85907 non-null  object        
 4   Customer Remarks      85907 non-null  object        
 5   Order_id              85907 non-null  object        
 6   Survey_response_Date  85907 non-null  datetime64[ns]
 7   Customer_City         85907 non-null  object        
 8   Agent_name            85907 non-null  object        
 9   Supervisor            85907 non-null  object        
 10  Manager               85907 non-null  object        
 11  Tenure Bucket         85907 non-null  object        
 12  Agent Shift           85907 non-null  object        
 13  CSAT Score      

## 4.2 Encoding data kategorikal dengan one hot encoding

In [123]:
fitur_kategori = [
    'channel_name', 'category', 'Sub-category', 'Customer_City',
    'Agent_name', 'Supervisor', 'Manager',
    'Tenure Bucket', 'Agent Shift'
]
X = pd.get_dummies(data[fitur_kategori])
y = data['CSAT_Label']
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85907 entries, 0 to 85906
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Unique id             85907 non-null  object        
 1   channel_name          85907 non-null  object        
 2   category              85907 non-null  object        
 3   Sub-category          85907 non-null  object        
 4   Customer Remarks      85907 non-null  object        
 5   Order_id              85907 non-null  object        
 6   Survey_response_Date  85907 non-null  datetime64[ns]
 7   Customer_City         85907 non-null  object        
 8   Agent_name            85907 non-null  object        
 9   Supervisor            85907 non-null  object        
 10  Manager               85907 non-null  object        
 11  Tenure Bucket         85907 non-null  object        
 12  Agent Shift           85907 non-null  object        
 13  CSAT Score      

# 5.Split data dan prediksi

In [124]:
# Membagi data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Membuat dan melatih model Naive Bayes
model = GaussianNB()
model.fit(X_train, y_train)

# Prediksi
y_pred = model.predict(X_test)

# 6. Evaluasi Model

In [125]:
# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)

# Accuracy
acc = accuracy_score(y_test, y_pred)
print("Accuracy:", acc)

# Precision
prec = precision_score(y_test, y_pred, pos_label="High")
print("Precision (High as positive):", prec)

# Recall
rec = recall_score(y_test, y_pred, pos_label="High")
print("Recall (High as positive):", rec)

# F1 Score
f1 = f1_score(y_test, y_pred, pos_label="High")
print("F1 Score (High as positive):", f1)

# Report Lengkap
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Confusion Matrix:
 [[ 1099 13112]
 [  186  2785]]
Accuracy: 0.2260505179839367
Precision (High as positive): 0.8552529182879377
Recall (High as positive): 0.07733445922172964
F1 Score (High as positive): 0.14184305627258648

Classification Report:
               precision    recall  f1-score   support

        High       0.86      0.08      0.14     14211
         Low       0.18      0.94      0.30      2971

    accuracy                           0.23     17182
   macro avg       0.52      0.51      0.22     17182
weighted avg       0.74      0.23      0.17     17182

